# Gene expression data

In [98]:
import pandas as pd
import numpy as np

First, let's import the gene expression data.

In [3]:
gasch_data = pd.read_csv("complete_dataset_gasch.txt", sep="\t")
gasch_data = gasch_data.iloc[:,:11] # Filter only heat-shock data from first experiment (hs-1).
gasch_data

,UID,NAME,GWEIGHT,Heat Shock 05 minutes hs-1,Heat Shock 10 minutes hs-1,Heat Shock 15 minutes hs-1,Heat Shock 20 minutes hs-1,Heat Shock 30 minutes hs-1,Heat Shock 40 minutes hs-1,Heat Shock 60 minutes hs-1,Heat Shock 80 minutes hs-1
0,YAL001C,YAL001C TFC3 TRANSCRIPTION ...,1,1.53,-0.06,0.58,0.52,0.42,0.16,0.79,NaN
1,YAL002W,YAL002W VPS8 VACUOLAR PROTEIN TARGETIN...,1,-0.01,-0.30,0.23,0.01,-0.15,0.45,-0.04,0.14
2,YAL003W,YAL003W EFB1 PROTEIN SYNTHESIS ...,1,0.15,-0.07,-0.25,-0.30,-1.12,-0.67,-0.15,-0.43
3,YAL004W,YAL004W UNKNOWN ...,1,0.24,0.76,0.20,0.34,0.11,0.07,0.01,0.36
4,YAL005C,YAL005C SSA1 ER AND MITOCHONDRIAL TRAN...,1,2.85,3.34,NaN,NaN,NaN,NaN,NaN,NaN
5,YAL007C,YAL007C ERP2 MEMBRANE TRAFFICKING; SEC...,1,-0.22,-0.12,-0.29,-0.51,-0.81,-0.47,0.28,-0.10
6,YAL008W,YAL008W FUN14 UNKNOWN ...,1,0.19,0.25,0.69,0.34,0.65,0.48,NaN,NaN
7,YAL009W,YAL009W SPO7 MEIOSIS ...,1,0.23,0.05,0.18,-0.15,-0.06,-0.19,-0.20,NaN
8,YAL010C,YAL010C MDM10 MITOCHONDRIAL BIOGENESIS ...,1,0.03,-0.23,0.33,NaN,0.23,-0.20,NaN,0.29
9,YAL011W,YAL011W UNKNOWN ...,1,0.01,-0.12,0.00,-0.22,NaN,-0.34,0.45,-0.10


From the Gasch website we have this description:

*"The data contained in all files represents the normalized, background-corrected log2 values of the Red/Green ratios measured on the DNA microarrays. Data from the figures have been mathematically transformed as described in Materials and Methods, such that each sample described is compared to the unstressed cells. The untransformed data can be accessed in the Complete Dataset file, with descriptions of each microarray reference in the online version of Materials and Methods. "*

I need to read the Materials and Methods of the Gasch paper again, but I think what they report is log2(expression at time t) - log2(expression at time 0). 

We have expression data for 6152 genes, although there seem to be quite a few missing datapoints. Note that all the genes are referred to as ORFs (e.g. YAL001C), not the common gene names. This does not match the format of the Yeastract data for the gene regulatory network, so we'll have to map the ORFs to the gene names.

I looked around a little bit and it seems that the Yeastract website has a way to do this, but we need to input a list of ORFs to get the gene names and viceversa. Because of this, I'll output the full list of names to a text file that we can hopefully just copy-paste to get the gene names.

In [4]:
outfile = open("ORFnames.txt", "w")
for orfname in list(gasch_data["UID"]):
    outfile.write(orfname + "\n")
outfile.close()    

I then copy-pasted the contents of this file here (http://www.yeastract.com/formorftogene.php) to get the list of common gene names. This mostly worked (except for a few gene names (~25) which are unidentified). That doesn't seem too bad since we have like 6100 genes total and we may even be able to hunt them down individually. However, there's  a problem: the output is an html table that is annoying to use. I couldn't figure out how to copy-paste it into LibreOffice Calc (a free version of Excel) in a way that the last two columns don't disappear. Basically, we need to figure out how to get the data out of there and into a csv or some sane format. It may also be possible to look at the source code for the html and try to get the data from there.

Alternatively, I also found this (https://www.uniprot.org/docs/yeast.txt), which is a list that maps all yeast ORFs to gene names (and lists when there's multiple gene names). We could also try writing a small script to get the name for each ORF. I'd rather use the Yeastract list if we can, since it's likely to match the names they have in the network data. 

After doing this we should be able to map the gene expression data to the connectivity data.

# Network connectivity data

There's a few different options we can use on the Yeastract website to construct our network. I think it's probably best if we design the way to get our adjacency matrix (or matrices).

In [62]:
yeastract_data = pd.read_csv("dna_only_yeastnet_full.csv", sep= ";")
tf_list = list(yeastract_data['6855'])
gene_list = list(yeastract_data.columns.values)[1:]

yeastract_data = yeastract_data.rename(index=yeastract_data['6855'])
yeastract_data = yeastract_data.iloc[:, 1:]

In [63]:
yeastract_data

,PAU8,SEO1,YAL066W,YAL065C,YAL064W-B,TDA8,YAL064W,FLO9,YBR089W,NHP6b,...,YER145C-A,YER190C-B,YGL194C-A,YIL102C-A,YJL077W-A,YKL023C-A,YKL065W-A,YPR010C-A,YPR145C-A,tX(XXX)L
Mal63p,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Oaf1p,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Pdr3p,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Hap3p,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Tod6p,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Sef1p,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Rtg3p,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Eds1p,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Reb1p,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Nrg2p,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
def tf_to_gene(tf_name):
    if tf_name[-1] == 'p':
        return tf_name.upper()[:-1]
    else:
        return tf_name

In [64]:
tf_list = [tf_to_gene(tf) for tf in tf_list]

In [32]:
outfile = open("Yeastract_ORFnames.txt", "w")
for gene in gene_list:
    outfile.write(gene + "\n")
outfile.close()

In [33]:
gene_list

['PAU8',
 'SEO1',
 'YAL066W',
 'YAL065C',
 'YAL064W-B',
 'TDA8',
 'YAL064W',
 'FLO9',
 'YBR089W',
 'NHP6b',
 'YBR090C',
 'TIM12',
 'PHO3',
 'PHO5',
 'PBY1',
 'RXT2',
 'YBR096W',
 'VPS15',
 'MMS4',
 'YBR099C',
 'FES1',
 'EXO84',
 'SIF2',
 'YBR103C-A',
 'YMC2',
 'VID24',
 'PHO88',
 'IML3',
 'AIM3',
 'CMD1',
 'ALG1',
 'YSA1',
 'CYC8',
 'YBR113W',
 'RAD16',
 'LYS2',
 'YBR116C',
 'TKL2',
 'TEF2',
 'MUD1',
 'CBP6',
 'GRS1',
 'MRPL36',
 'TFC1',
 'YBR124W',
 'PTC4',
 'TPS1',
 'VMA2',
 'ATG14',
 'OPY1',
 'SHE3',
 'CCZ1',
 'AGP2',
 'HSL7',
 'YBR134W',
 'CKS1',
 'MEC1',
 'YBR137W',
 'YBR138C',
 'YBR139W',
 'IRA1',
 'BMT2',
 'MAK5',
 'SUP45',
 'YBR144C',
 'ADH5',
 'MRPS9',
 'RTC2',
 'YSW1',
 'ARA1',
 'TBS1',
 'APD1',
 'SPP381',
 'RIB7',
 'RPB5',
 'CNS1',
 'SLI15',
 'ICS2',
 'AMN1',
 'IFA38',
 'CDC28',
 'TOS1',
 'YSY6',
 'EXO5',
 'ARL1',
 'UBS1',
 'TYR1',
 'POP7',
 'PEX32',
 'SSE2',
 'NPL4',
 'SEC66',
 'SMY2',
 'UMP1',
 'YBR174C',
 'SWD3',
 'ECM31',
 'EHT1',
 'YBR178W',
 'FZO1',
 'DTR1',
 'RPS6b',


In [43]:
gene_to_orf_df = pd.read_csv("tftoorf.csv")
gene_to_orf_df

,Inserted List,Unnamed: 1,ORF Name,Gene Name
0,PAU8,NaN,YAL068C,PAU8
1,SEO1,NaN,YAL067C,SEO1
2,YAL066W,NaN,YAL066W,YAL066W
3,YAL065C,NaN,YAL065C,YAL065C
4,YAL064W-B,NaN,YAL064W-B,YAL064W-B
5,TDA8,NaN,YAL064C-A,TDA8
6,YAL064W,NaN,YAL064W,YAL064W
7,FLO9,NaN,YAL063C,FLO9
8,YBR089W,NaN,YBR089W,YBR089W
9,NHP6b,NaN,YBR089C-A,NHP6b


In [84]:
gene_to_orf = dict(zip(list(gene_to_orf_df.iloc[:,0]), list(gene_to_orf_df.iloc[:,2])))

In [85]:
gene_to_orf

{'PAU8': 'YAL068C',
 'SEO1': 'YAL067C',
 'YAL066W': 'YAL066W',
 'YAL065C': 'YAL065C',
 'YAL064W-B': 'YAL064W-B',
 'TDA8': 'YAL064C-A',
 'YAL064W': 'YAL064W',
 'FLO9': 'YAL063C',
 'YBR089W': 'YBR089W',
 'NHP6b': 'YBR089C-A',
 'YBR090C': 'YBR090C',
 'TIM12': 'YBR091C',
 'PHO3': 'YBR092C',
 'PHO5': 'YBR093C',
 'PBY1': 'YBR094W',
 'RXT2': 'YBR095C',
 'YBR096W': 'YBR096W',
 'VPS15': 'YBR097W',
 'MMS4': 'YBR098W',
 'YBR099C': 'YBR099C',
 'FES1': 'YBR101C',
 'EXO84': 'YBR102C',
 'SIF2': 'YBR103W',
 'YBR103C-A': 'YBR103C-A',
 'YMC2': 'YBR104W',
 'VID24': 'YBR105C',
 'PHO88': 'YBR106W',
 'IML3': 'YBR107C',
 'AIM3': 'YBR108W',
 'CMD1': 'YBR109C',
 'ALG1': 'YBR110W',
 'YSA1': 'YBR111C',
 'CYC8': 'YBR112C',
 'YBR113W': 'YBR113W',
 'RAD16': 'YBR114W',
 'LYS2': 'YBR115C',
 'YBR116C': 'YBR116C',
 'TKL2': 'YBR117C',
 'TEF2': 'YBR118W',
 'MUD1': 'YBR119W',
 'CBP6': 'YBR120C',
 'GRS1': 'YBR121C',
 'MRPL36': 'YBR122C',
 'TFC1': 'YBR123C',
 'YBR124W': 'YBR124W',
 'PTC4': 'YBR125C',
 'TPS1': 'YBR126C',
 'V

In [86]:
yeastract_data

,PAU8,SEO1,YAL066W,YAL065C,YAL064W-B,TDA8,YAL064W,FLO9,YBR089W,NHP6b,...,YER145C-A,YER190C-B,YGL194C-A,YIL102C-A,YJL077W-A,YKL023C-A,YKL065W-A,YPR010C-A,YPR145C-A,tX(XXX)L
Mal63p,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Oaf1p,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Pdr3p,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Hap3p,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Tod6p,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Sef1p,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Rtg3p,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Eds1p,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Reb1p,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Nrg2p,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [87]:
yeastract_data

,PAU8,SEO1,YAL066W,YAL065C,YAL064W-B,TDA8,YAL064W,FLO9,YBR089W,NHP6b,...,YER145C-A,YER190C-B,YGL194C-A,YIL102C-A,YJL077W-A,YKL023C-A,YKL065W-A,YPR010C-A,YPR145C-A,tX(XXX)L
Mal63p,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Oaf1p,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Pdr3p,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Hap3p,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Tod6p,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Sef1p,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Rtg3p,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Eds1p,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Reb1p,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Nrg2p,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [88]:
rownames = [gene_to_orf[tf] for tf in tf_list]
rownames

['MALR',
 'YAL051W',
 'YBL005W',
 'YBL021C',
 'YBL054W',
 'YBL066C',
 'YBL103C',
 'YBR033W',
 'YBR049C',
 'YBR066C',
 'YBR083W',
 'YBR112C',
 'YBR150C',
 'YBR182C',
 'YBR239C',
 'YBR240C',
 'YBR297W',
 'YCL055W',
 'YCL058C',
 'YCR018C',
 'YCR065W',
 'YCR084C',
 'YCR106W',
 'YDL020C',
 'YDL048C',
 'YDL056W',
 'YDL106C',
 'YDL170W',
 'YDL233W',
 'YDR009W',
 'YDR034C',
 'YDR043C',
 'YDR081C',
 'YDR096W',
 'YDR123C',
 'YDR146C',
 'YDR169C',
 'YDR174W',
 'YDR176W',
 'YDR207C',
 'YDR213W',
 'YDR216W',
 'YDR253C',
 'YDR259C',
 'YDR277C',
 'YDR310C',
 'YDR421W',
 'YDR423C',
 'YDR451C',
 'YDR463W',
 'YDR480W',
 'YDR501W',
 'YDR520C',
 'YEL007W',
 'YEL009C',
 'YER028C',
 'YER040W',
 'YER045C',
 'YER068W',
 'YER088C',
 'YER109C',
 'YER111C',
 'YER130C',
 'YER169W',
 'YFL021W',
 'YFL031W',
 'YFL052W',
 'YFR034C',
 'YGL013C',
 'YGL035C',
 'YGL071W',
 'YGL073W',
 'YGL096W',
 'YGL162W',
 'YGL166W',
 'YGL209W',
 'YGL237C',
 'YGL254W',
 'YGR044C',
 'YGR067C',
 'YGR249W',
 'YGR288W',
 'YHL009C',
 'YHL02

In [92]:
# Change to TF if possible, otherwise keep same name.
colnames = [gene_to_orf.get(gene, gene) for gene in gene_list]

In [101]:
for gene in gene_list:
    if gene_to_orf.get(gene, None) == None:
        print(gene)

OCT1
MAY24


In [93]:
colnames

['YAL068C',
 'YAL067C',
 'YAL066W',
 'YAL065C',
 'YAL064W-B',
 'YAL064C-A',
 'YAL064W',
 'YAL063C',
 'YBR089W',
 'YBR089C-A',
 'YBR090C',
 'YBR091C',
 'YBR092C',
 'YBR093C',
 'YBR094W',
 'YBR095C',
 'YBR096W',
 'YBR097W',
 'YBR098W',
 'YBR099C',
 'YBR101C',
 'YBR102C',
 'YBR103W',
 'YBR103C-A',
 'YBR104W',
 'YBR105C',
 'YBR106W',
 'YBR107C',
 'YBR108W',
 'YBR109C',
 'YBR110W',
 'YBR111C',
 'YBR112C',
 'YBR113W',
 'YBR114W',
 'YBR115C',
 'YBR116C',
 'YBR117C',
 'YBR118W',
 'YBR119W',
 'YBR120C',
 'YBR121C',
 'YBR122C',
 'YBR123C',
 'YBR124W',
 'YBR125C',
 'YBR126C',
 'YBR127C',
 'YBR128C',
 'YBR129C',
 'YBR130C',
 'YBR131W',
 'YBR132C',
 'YBR133C',
 'YBR134W',
 'YBR135W',
 'YBR136W',
 'YBR137W',
 'YBR138C',
 'YBR139W',
 'YBR140C',
 'YBR141C',
 'YBR142W',
 'YBR143C',
 'YBR144C',
 'YBR145W',
 'YBR146W',
 'YBR147W',
 'YBR148W',
 'YBR149W',
 'YBR150C',
 'YBR151W',
 'YBR152W',
 'YBR153W',
 'YBR154C',
 'YBR155W',
 'YBR156C',
 'YBR157C',
 'YBR158W',
 'YBR159W',
 'YBR160W',
 'YBR162C',
 'YBR162

In [83]:
gene_list

['PAU8',
 'SEO1',
 'YAL066W',
 'YAL065C',
 'YAL064W-B',
 'TDA8',
 'YAL064W',
 'FLO9',
 'YBR089W',
 'NHP6b',
 'YBR090C',
 'TIM12',
 'PHO3',
 'PHO5',
 'PBY1',
 'RXT2',
 'YBR096W',
 'VPS15',
 'MMS4',
 'YBR099C',
 'FES1',
 'EXO84',
 'SIF2',
 'YBR103C-A',
 'YMC2',
 'VID24',
 'PHO88',
 'IML3',
 'AIM3',
 'CMD1',
 'ALG1',
 'YSA1',
 'CYC8',
 'YBR113W',
 'RAD16',
 'LYS2',
 'YBR116C',
 'TKL2',
 'TEF2',
 'MUD1',
 'CBP6',
 'GRS1',
 'MRPL36',
 'TFC1',
 'YBR124W',
 'PTC4',
 'TPS1',
 'VMA2',
 'ATG14',
 'OPY1',
 'SHE3',
 'CCZ1',
 'AGP2',
 'HSL7',
 'YBR134W',
 'CKS1',
 'MEC1',
 'YBR137W',
 'YBR138C',
 'YBR139W',
 'IRA1',
 'BMT2',
 'MAK5',
 'SUP45',
 'YBR144C',
 'ADH5',
 'MRPS9',
 'RTC2',
 'YSW1',
 'ARA1',
 'TBS1',
 'APD1',
 'SPP381',
 'RIB7',
 'RPB5',
 'CNS1',
 'SLI15',
 'ICS2',
 'AMN1',
 'IFA38',
 'CDC28',
 'TOS1',
 'YSY6',
 'EXO5',
 'ARL1',
 'UBS1',
 'TYR1',
 'POP7',
 'PEX32',
 'SSE2',
 'NPL4',
 'SEC66',
 'SMY2',
 'UMP1',
 'YBR174C',
 'SWD3',
 'ECM31',
 'EHT1',
 'YBR178W',
 'FZO1',
 'DTR1',
 'RPS6b',


In [102]:
final_data = pd.DataFrame(np.array(yeastract_data), index=rownames, columns=colnames)

In [109]:
print(colnames)

['YAL068C', 'YAL067C', 'YAL066W', 'YAL065C', 'YAL064W-B', 'YAL064C-A', 'YAL064W', 'YAL063C', 'YBR089W', 'YBR089C-A', 'YBR090C', 'YBR091C', 'YBR092C', 'YBR093C', 'YBR094W', 'YBR095C', 'YBR096W', 'YBR097W', 'YBR098W', 'YBR099C', 'YBR101C', 'YBR102C', 'YBR103W', 'YBR103C-A', 'YBR104W', 'YBR105C', 'YBR106W', 'YBR107C', 'YBR108W', 'YBR109C', 'YBR110W', 'YBR111C', 'YBR112C', 'YBR113W', 'YBR114W', 'YBR115C', 'YBR116C', 'YBR117C', 'YBR118W', 'YBR119W', 'YBR120C', 'YBR121C', 'YBR122C', 'YBR123C', 'YBR124W', 'YBR125C', 'YBR126C', 'YBR127C', 'YBR128C', 'YBR129C', 'YBR130C', 'YBR131W', 'YBR132C', 'YBR133C', 'YBR134W', 'YBR135W', 'YBR136W', 'YBR137W', 'YBR138C', 'YBR139W', 'YBR140C', 'YBR141C', 'YBR142W', 'YBR143C', 'YBR144C', 'YBR145W', 'YBR146W', 'YBR147W', 'YBR148W', 'YBR149W', 'YBR150C', 'YBR151W', 'YBR152W', 'YBR153W', 'YBR154C', 'YBR155W', 'YBR156C', 'YBR157C', 'YBR158W', 'YBR159W', 'YBR160W', 'YBR162C', 'YBR162W-A', 'YBR163W', 'YBR164C', 'YBR165W', 'YBR166C', 'YBR167C', 'YBR168W', 'YBR169C',

In [108]:
final_data

,YAL068C,YAL067C,YAL066W,YAL065C,YAL064W-B,YAL064C-A,YAL064W,YAL063C,YBR089W,YBR089C-A,...,YER145C-A,YER190C-B,YGL194C-A,YIL102C-A,YJL077W-A,YKL023C-A,YKL065W-A,YPR010C-A,YPR145C-A,tX(XXX)L
MALR,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
YAL051W,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
YBL005W,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
YBL021C,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
YBL054W,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
YBL066C,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
YBL103C,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
YBR033W,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
YBR049C,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
YBR066C,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [122]:
cols=list(final_data.columns)
rows=list(final_data.index)
d = []
for x in cols:
    if x in rows:
        d.append(list(final_data.loc[x]))
    else:
        d.append([0]*len(cols))
A = np.array(d)
A = pd.DataFrame(A, index=cols, columns=cols)
#corrected_matrix=df.transpose()

In [123]:
A

,YAL068C,YAL067C,YAL066W,YAL065C,YAL064W-B,YAL064C-A,YAL064W,YAL063C,YBR089W,YBR089C-A,...,YER145C-A,YER190C-B,YGL194C-A,YIL102C-A,YJL077W-A,YKL023C-A,YKL065W-A,YPR010C-A,YPR145C-A,tX(XXX)L
YAL068C,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
YAL067C,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
YAL066W,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
YAL065C,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
YAL064W-B,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
YAL064C-A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
YAL064W,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
YAL063C,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
YBR089W,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
YBR089C-A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [119]:
corrected_df = A

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [124]:
A.loc["YBR182C"]

YAL068C      1
YAL067C      0
YAL066W      0
YAL065C      1
YAL064W-B    0
YAL064C-A    0
YAL064W      0
YAL063C      1
YBR089W      0
YBR089C-A    0
YBR090C      0
YBR091C      0
YBR092C      0
YBR093C      0
YBR094W      0
YBR095C      0
YBR096W      0
YBR097W      0
YBR098W      0
YBR099C      0
YBR101C      0
YBR102C      0
YBR103W      0
YBR103C-A    0
YBR104W      0
YBR105C      0
YBR106W      0
YBR107C      0
YBR108W      0
YBR109C      0
            ..
YBL100W-A    0
YBL100W-B    0
YBL100W-C    0
YCL058W-A    0
YDR524W-C    0
YLR157W-D    0
YLR157W-E    0
YNL097C-B    0
RTM1         0
MALR         0
ICR1         0
PWR1         0
YDR119W-A    0
YDR374W-A    0
YBR126W-A    0
YBR201C-A    0
YCL048W-A    0
YDL007C-A    0
YDL022C-A    0
YDR461C-A    0
YER145C-A    0
YER190C-B    0
YGL194C-A    0
YIL102C-A    0
YJL077W-A    0
YKL023C-A    0
YKL065W-A    0
YPR010C-A    0
YPR145C-A    0
tX(XXX)L     0
Name: YBR182C, Length: 6855, dtype: int64

In [ ]:
def get_adjacency_matrix(yeastract_csv_filename, gene_to_orf_df):
    def tf_to_gene(tf_name):
    if tf_name[-1] == 'p':
        return tf_name.upper()[:-1]
    else:
        return tf_name
    
    yeastract_data = pd.read_csv(yeastract_csv_filename, sep= ";")
    tf_list = list(yeastract_data['6855'])
    gene_list = list(yeastract_data.columns.values)[1:]
    
    # Fix dataframe so it has right things as rownames.
    yeastract_data = yeastract_data.rename(index=yeastract_data['6855'])
    yeastract_data = yeastract_data.iloc[:, 1:]
    
    gene_to_orf = dict(zip(list(gene_to_orf_df.iloc[:,0]), list(gene_to_orf_df.iloc[:,2])))
    
    tf_list = [tf_to_gene(tf) for tf in tf_list]